## Proceso de ETL en el dataset australian_user_items

#### Este proceso se realiza con la finalidad de leer el dataset con el formato correcto, eliminar las columnas que no se necesitan para responder las consultas y hacer las transformaciones necesarias para optimizar el rendimiento de la API y el entrenamiento del modelo de aprendizaje automático.

In [9]:
import pandas as pd
import json
import ast

#### El archivo JSON se carga en un DataFrame utilizando la función open() de Python para abrir el archivo y leer su contenido en una lista de cadenas de texto. Luego, utiliza una comprensión de lista para evaluar cada cadena de texto como una expresión de Python y convertirla en un diccionario.

In [10]:
data_list = []
file_path = "datasets_originales/australian_users_items.json"
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError:
            continue

### Se crea un DataFrame de pandas a partir de la lista de diccionarios.

In [11]:
df_items = pd.DataFrame(data_list)

### Visualización del DataFrame.

In [12]:
df_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### Se utiliza una función auxiliar para extraer los pares clave/valor de los diccionarios y crear las nuevas columnas de manera más eficiente.

In [13]:
def extract_value(x, key):
    if len(x) > 0:
        return x[0].get(key, 0)
    return 0

df_items['playtime_forever'] = df_items['items'].apply(lambda x: extract_value(x, 'playtime_forever'))
df_items['playtime_2weeks'] = df_items['items'].apply(lambda x: extract_value(x, 'playtime_2weeks'))
df_items['id'] = df_items['items'].apply(lambda x: extract_value(x, 'item_id'))
df_items[['playtime_forever', 'playtime_2weeks']] = df_items[['playtime_forever', 'playtime_2weeks']].astype(int)


### Se eliminan columnas específicas del DataFrame con el objetivo de reducir su tamaño y mejorar su eficiencia.

In [14]:
df_items.drop(['user_url'], axis=1, inplace=True)  
df_items.drop(['items'], axis=1, inplace=True)

### Se visualiza el DataFrame para confirmar si muestra los resultados deseados. 

In [15]:
df_items

,user_id,items_count,steam_id,playtime_forever,playtime_2weeks,id
0,76561197970982479,277,76561197970982479,6,0,10
1,js41637,888,76561198035864385,0,0,10
2,evcentric,137,76561198007712555,923,0,1200
3,Riot-Punch,328,76561197963445855,0,0,10
4,doctr,541,76561198002099482,1131,0,300
...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,0,0,413850
88306,76561198326700687,177,76561198326700687,0,0,11020
88307,XxLaughingJackClown77xX,0,76561198328759259,0,0,0
88308,76561198329548331,7,76561198329548331,677,677,304930


### El DataFrame obtenido de los diferentes procesos se exporta a un archivo CSV.

In [16]:
df_items.to_csv('australian_users_items_clean.csv', index=False)